## CNN Using nn.Sequential

In [ ]:
import torch
import torch.nn as nn

# Create a sequential layer
model1 = nn.Sequential(

    # First convolutional layer
    nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Second convolutional layer
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Flatten(),

    nn.Linear(32 * 7 * 7, 10)
)

In [ ]:
# Testing the model
x1 = torch.randn(64, 1, 28, 28)
output1 = model1(x1)
print(output1.size())

torch.Size([64, 10])


## Equivalent CNN Using nn.Module

In [ ]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second convolutional block
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16 * 8 * 8, 10)

    def forward(self, x):
        # Apply the first convolutional block
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # Apply the second convolutional block
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.flatten(x)
        x = self.fc(x)
        return x

In [ ]:
# Train this model on the CIFAR10 dataset
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 170M/170M [00:11<00:00, 14.5MB/s]


In [ ]:
# Instantiate and train the model
model2 = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

#Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model2(images) # calculate the output
        loss = criterion(outputs, labels) # calculate the loss

        # Backward pass and optimization
        optimizer.zero_grad() # reset the gradients
        loss.backward() # initiates the backpropagation process
        optimizer.step() # update model parameters

        running_loss += loss.item() #update the loss
        _, predicted = torch.max(outputs.data, 1) # retrieve the index of max
        total += labels.size(0) # update the total
        correct += (predicted == labels).sum().item() # update the correct
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/total_step:.4f}, Accuracy: {100*correct/total:.2f}%')

# Test the model
model2.eval() # Sets the model in evaluation mode
with torch.no_grad(): # disable gradient calculation
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device) # move data to device
        labels = labels.to(device) # move data to device
        outputs = model2(images) # forward pass
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) # update the total
        correct += (predicted == labels).sum().item() # update the correct

    print(f'Test Accuracy: {100 * correct / total:.2f}%') # print the results

# Save the model
torch.save(model2.state_dict(), 'simple_cnn.pth')

Epoch [1/10], Loss: 1.5301, Accuracy: 45.61%
Epoch [2/10], Loss: 1.2320, Accuracy: 56.56%
Epoch [3/10], Loss: 1.1135, Accuracy: 61.11%
Epoch [4/10], Loss: 1.0425, Accuracy: 63.74%
Epoch [5/10], Loss: 0.9958, Accuracy: 65.31%
Epoch [6/10], Loss: 0.9615, Accuracy: 66.37%
Epoch [7/10], Loss: 0.9340, Accuracy: 67.41%
Epoch [8/10], Loss: 0.9153, Accuracy: 68.13%
Epoch [9/10], Loss: 0.8985, Accuracy: 68.78%
Epoch [10/10], Loss: 0.8836, Accuracy: 69.25%
Test Accuracy: 66.44%


In [ ]:
#pip install tensorboard

In [ ]:
# Visualize in Tensorboard
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter("run/simple_cnn")
dummy_input = torch.randn(64, 3, 32, 32) # mini-batch of 10 images
writer.add_graph(model2, dummy_input)
writer.close()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir run/simple_cnn

In [ ]:
!pip install pyngrok

In [ ]:
# Google Colab runs in a cloud VM, so localhost:6006 is inaccessible. Instead, use ngrok to tunnel.
from pyngrok import ngrok

In [ ]:
LOG_DIR = "run/simple_cnn"

# Kill previous TB instances
!pkill tensorboard

# Start TensorBoard
get_ipython().system_raw(
    f'tensorboard --logdir {LOG_DIR} --host 0.0.0.0 --port 6006 &'
)

from google.colab import userdata
auth_token = userdata.get('AUTH_TOKEN')

# Open ngrok tunnel to TensorBoard
!ngrok authtoken {auth_token}
tb_url = ngrok.connect(6006)
print(f"🔗 TensorBoard is live at: {tb_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🔗 TensorBoard is live at: NgrokTunnel: "https://88e4-35-229-230-199.ngrok-free.app" -> "http://localhost:6006"
